<a href="https://colab.research.google.com/github/HiagoAD/GDI2/blob/master/Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import numpy as np
import pandas as pd

dengue = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/dengue.csv", encoding ="UTF-8")
enlurb = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/emlurb.csv", encoding ="UTF-8")

minha_lista = [1,2,3,4,5]
a = np.array(minha_lista)
a

np.arange(0,10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [30]:
dengue["no_bairro_residencia"] = dengue["no_bairro_residencia"].astype('category')
dengue["no_bairro_residencia"].value_counts()

IBURA                186
COHAB                 63
AREIAS                31
BOA VIAGEM            31
AFOGADOS              31
                    ... 
JARDIM PETROPOLIS      1
JOANA BEZERRA          1
POCO DA PANELA         1
JORDAO BAIXO           1
DERBY                  1
Name: no_bairro_residencia, Length: 95, dtype: int64

In [32]:
enlurb.head()

,_id,GRUPOSERVICO_CODIGO,GRUPOSERVICO_DESCRICAO,SERVICO_CODIGO,SERVICO_DESCRICAO,LOGRADOURO,NUMERO,BAIRRO,RPA,DATA_DEMANDA,SITUACAO,DATA_ULT_SITUACAO
0,1,7,ARBORIZAÇÃO,8,PODA DE ARVORES,1TR JOSE MENEZES DE LYRA FILHO,34,DOIS UNIDOS,2,2020-07-08T00:00:00,ATENDIDA,2020-09-23T00:00:00
1,2,7,ARBORIZAÇÃO,8,PODA DE ARVORES,1TR VIRGILIO CAMPELO,85,MADALENA,4,2020-07-30T00:00:00,ATENDIDA,2020-08-07T00:00:00
2,3,7,ARBORIZAÇÃO,8,PODA DE ARVORES,1TR SANTA LUCIA,412,IPUTINGA,4,2020-09-10T00:00:00,ATENDIDA,2020-11-10T00:00:00
3,4,7,ARBORIZAÇÃO,8,PODA DE ARVORES,2TR REALEZA,776,SAO JOSE,1,2020-10-08T00:00:00,ATENDIDA,2020-12-10T00:00:00
4,5,7,ARBORIZAÇÃO,8,PODA DE ARVORES,2TR REALEZA,776,SAO JOSE,1,2020-12-14T00:00:00,ATENDIDA,2020-12-14T00:00:00


In [40]:
enlurb["BAIRRO"] = enlurb["BAIRRO"].astype('category')
enlurb['GRUPOSERVICO_DESCRICAO'] = enlurb['GRUPOSERVICO_DESCRICAO'].astype('category')
enlurb[enlurb['GRUPOSERVICO_DESCRICAO'] == 'DRENAGEM']['BAIRRO'].value_counts()

SANTO AMARO             436
BOA VIAGEM              382
AFOGADOS                328
CORDEIRO                325
SAO JOSE                290
                       ... 
CIDADE UNIVERSITARIA      6
JAQUEIRA                  5
SOLEDADE                  3
PEIXINHOS                 3
PAU FERRO                 0
Name: BAIRRO, Length: 94, dtype: int64